
<img align="left" src = https://www.linea.org.br/wp-content/themes/LIneA/imagens/logo-header.png width=100 style="padding: 30px"> 
<img align="left" src = https://cdn2.webdamdb.com/1280_c3PXjCZbPM23.png width=180> <!-- style="padding: 20px"--> 

# Training Set Maker - Tutorial Notebook

**Contact author**: Julia Gschwend ([julia@linea.org.br](mailto:julia@linea.org.br)) 

**Last verified run**: yyyy-mm-dd <br>



# Notebook under construction - please wait for the first verified version

Photo-z (PZ) Training Set Maker (TSM) - [pz-tsm](https://github.com/linea-it/pz-tsm) - is tool available in the PZ Server library to support the creation of Training Sets for catalog-level machine-learning-based photo-z algorithms using spec-z (or true z, for simulations) catalogs provided by the [PZ Server](https://github.com/linea-it/pz-server) and photometric data from LSST Objects Catalogs. 

# classes para prod types - specz cat, training set (herda do pd.dataframe) 
# pz server traz objetos já na classe correspondente
### atributos: prod id, metadata
# tsm tem funções 
### combine specz catalogs - recebe lista de objetos specz catalogs ou lista de prod ids e criterios como argumento
### matching - recebe 1 specz e 1 photo set (datafranme qualquer local) ou 1 prod id e 1 release tag (job remoto)
***


#### Setup

In [1]:
from pz_server import PzServer
from pz_tsm import SpeczSample
#from train_valid import TrainValid

In [1]:
-    "Training and Validation (or Test) Sets are the product of matching (spatially) a given Spec-z Catalog (single survey or compilation) to the photometric data
, in this case, the LSST Objects Catalog. In fact, in most cases, the Training and Validation Sets are just the result of splitting the product of matching into t
wo parts. Hence, the Training and Validation Sets are usually found together. In the case of simulations, the Training and Validation Sets can be just a selection
 from the simulated catalog that contains the true redshifts and the photometric data required to train and validate the photo-z algorithms.\n",
-    "\n",
-    "In the Photo-z Server, there is no dependency between these two products. Users can upload Training and Validation Sets separately, even though they are ver
y similar in format and contents. For each pair of Training and Validation Sets, the user will perform two uploads and, consequently, two new entries will be adde
d to the database. \n",
-    "\n",
-    "_Note 1: There is an ambiguity between the so-called Validation and Test sets found in the literature. In some cases, it is just a matter of terminology, an
d both play the same role: be used for computing the photo-z metrics as an independent sample from that used for training. In other cases, when the training proce
dure has a recursive optimization method, the three sets of Training/Validation/Test are distinct, and each one plays a different role. In the context of the PZ S
erver, there is no distiction between Validation or Test sets. The users are responsible for giving information on how to interpret the subsets on the description
 field._\n",

SyntaxError: EOL while scanning string literal (<ipython-input-1-560ac5ca9fe7>, line 1)

In [ ]:
#import sys
#sys.path.append('/Users/julia/github/pz-server-lib')

In [6]:
pz_server = PzServer(token="b9bd517b1effa3c649f8ff092992145d44197d89", host="pz-dev")

#### Get basic info from PZ Server

In [8]:
pz_server.display_product_types()

product_type,description
Spec-z Catalog,Catalog of spectroscopic redshifts and positions (usually equatorial coordinates).
Training Set,"Training set for photo-z algorithms (tabular data). It usually contains magnitudes, errors, and true redshifts."
Validation Set,"Validation/test set for photo-z algorithms (tabular data). It usually contains magnitudes, errors, and true redshifts."
Validation Results,Results of a photo-z validation procedure (free format). Usually contains photo-z estimates (single estimates and/or pdf) of a validation set and photo-z validation metrics.
Photo-z Table,"Results of a photo-z estimation procedure. If the data is larger than the file upload limit (200MB), the product entry stores only the metadata (instructions on accessing the data should be provided in the description field."


In [ ]:
pz_server.list_users

In [ ]:
pz_server.list_releases

In [ ]:
pz_server.list_products_available(user="PZ Coord. Group", product_type="specz_catalog", release="all")

## 1. Spec-z Catalog 
#### 1.1 List Spec-z Catalogs available on Pz Server

In [ ]:
pz_server.list_products_available(user="PZ Coord. Group", product_type="specz_catalog", release="all")

#### 1.2 Get metadata of a list of Spec-z Catalogs 

In [ ]:
metadata_specz_1 = PzServer.get_product_metadata(product_id="0001")
metadata_specz_1  # markdown table

In [ ]:
metadata_specz_2 = PzServer.get_product_metadata(product_id="0002")
metadata_specz_2  # markdown table

In [ ]:
metadata_specz_3 = PzServer.get_product_metadata(product_id="0003")
metadata_specz_3  # markdown table

#### 1.3 Create a single Spec-z Sample from a single Spec-z Catalogs

In [ ]:
specz_sample_one = SpeczSample(catalogs=["0001"])

In [ ]:
specz_sample_one.metadata # metadata

In [ ]:
specz_sample_one.data # dataframe

Apply filters on data

In [ ]:
# to do: pandas query

In [ ]:
# include info about filters used in the readme txt

Save file with filtered spec-z sample

In [ ]:
specz_sample_one.save_file() # save 2 files (data and readme), ready to upload on PZ Server

#### 1.4 Create a single Spec-z Sample from a list of Spec-z Catalogs

In [ ]:
specz_sample_all = SpeczSample(catalogs=["0001", "0002", "0003"], resolve_multiple="keep all")


In [ ]:
specz_sample_best = SpeczSample(catalogs=["0001", "0002", "0003"], resolve_multiple="best") # best flag, then smallest error


In [ ]:
specz_sample_newest = SpeczSample(catalogs=["0001", "0002", "0003"], resolve_multiple="newest") # newest survey

In [ ]:
specz_sample_best.metadata # combined metadata (markdown table)

In [ ]:
specz_sample_best.data # dataframe

Apply filters on data

In [ ]:
# to do: pandas query

Save file with filtered spec-z sample

In [ ]:
specz_sample_best.save_file(file_format="parquet") # save 2 files (data and readme(json)), ready to upload on PZ Server

Mini QA of combined Spec-z Sample

## 2. Training and Validation Sets

The training and Validation Sets are build as the combination of a Spec-z Sample (a Spec-z Sample object as defined above, or a Spec-z Catalog data product retrieved from the PZ Server) and photometric data retrieved from the API aspect of RSP.   

(to do: include case for 3 subsets (train/valid/test)
#### 2.1 Create Training and Validation Sets from a Spec-z Sample object

In [ ]:
split="random"
train_fraction=0.7 # use 1.0 or 0.0 to create training set or validation set separately
observables=["mag_u", "mag_g", "mag_r", "mag_i", "mag_z", "mag_y", 
             "magerr_u", "magerr_g", "magerr_r", "magerr_i", "magerr_z", "magerr_y"] # any list of columns from LSST objects catalog

In [ ]:
train_valid_local_specz = TrainValid(specz_sample_best, split=split, train_fraction=train_fraction, observables=observables)      

In [ ]:
train_valid_local_specz.train_metadata # markdown table

In [ ]:
train_valid_local_specz.train_data # dataframe

In [ ]:
train_valid_local_specz.valid_metadata # markdown table

In [ ]:
train_valid_local_specz.valid_data  # dataframe

Mini QA of Training and Validation Sets

Save files

In [ ]:
train_valid_local_specz.save_file(file_format="parquet")  # save 4 files (2 data and 2 readme (json)), ready to upload on PZ Server

#### 2.2 Create Training and Validation Sets from a Spec-z Catalog data product in PZ Server

In [ ]:
train_valid_remote_specz = TrainValid("0001", split=split, train_fraction=train_fraction, observables=observables)      

In [ ]:
train_valid_remote_specz.train_metadata

In [ ]:
train_valid_remote_specz.train_data # dataframe

In [ ]:
train_valid_remote_specz.valid_metadata

In [ ]:
train_valid_remote_specz.valid_data  # dataframe

Mini QA of Training and Validation Sets